In [ ]:
import torch
from torch import nn
from torchvision import models
import os
from glob import glob

device = "cuda" if torch.cuda.is_available() else "cpu"

In [ ]:
import urllib
import numpy as np
import requests
import cv2
import torchvision

import PIL
from PIL.Image import Image
from matplotlib import pyplot as plt
from torch.autograd import Variable

from src.interpretation.interpretation_functions import layer_cam_gen, guided_backprop_gen, gradpp_cam_gen, saliency_gen, grad_cam_gen
from src.interpretation.image_utils import im_show, preprocess, ClassifierOutputSoftmaxTarget, \
    confidence_change_apply_cam, \
    get_target_index, preprocess_image, view, to_plot_bbox
from src.interpretation.metrics import deletion_metric, deletion_game, preservation_metric, preservation_game, \
    average_drop_item, \
    avg_drop_list, increase_in_confidence_item, increase_in_confidence_list, topk_img, sparsity, iou_loc

Модель EfficientNetB0      
Deletion Game diffs=0.0012688534 perc=10.0  
Preservation Game diffs=0.00013 perc= 990.0
Avg Drop 81.57
Increase in Confidence  0.02  
Sparsity 5.0424   

Модель VGG      
Deletion Game diffs=0.12864499, perc=10.0  
Preservation Game diffs=0.06659404 perc= 990.0
Avg Drop 89.966
Increase in Confidence 0.05
Sparsity 3.39178  
Модель Resnet      
Deletion Game diffs=0.02668962 perc=1.0
Preservation Game diffs=0.01636001 perc= 990.0
Avg Drop 87.85
Increase in Confidence  0.03
Sparsity 2.5085 


In [ ]:
model = models.efficientnet_b0(weights="IMAGENET1K_V1").to(device)
model.eval()
transforms = models.EfficientNet_B0_Weights.IMAGENET1K_V1.transforms()

In [ ]:
import json
from matplotlib.patches import Rectangle
from matplotlib import patches
from PIL import Image, ImageFilter

with open("export-result (imagenett_FINAL).ndjson") as json_file:
    data = json.load(json_file)

list_iou = []

for i in range(len(data["a"])):
  name_dict = data["a"][i]["data_row"]["external_id"]
  
  bbox_dict_i = data["a"][i]["projects"][list(data["a"][i]["projects"].keys())[0]]["labels"][0]["annotations"]["objects"][0]["bounding_box"]
  bb_top_i = bbox_dict_i["top"]
  bb_left_i = bbox_dict_i["left"]
  bb_height_i = bbox_dict_i["height"]
  bb_width_i = bbox_dict_i["width"]
  im = Image.open(f"data_labeling_imagenet/{name_dict}")
  res, gray_res = guided_backprop_gen(model, 
                               im_show(f"data_labeling_imagenet/{name_dict}"),
                               target_layers= [model.features[-1]])
  bb_arr = np.array([bb_top_i, bb_left_i, bb_top_i + bb_height_i, bb_left_i + bb_width_i])
  iou_i = iou_loc(bb_arr, gray_res)
  list_iou.append(iou_i)

In [ ]:
print(f"IOU Average: {np.mean(list_iou)}")

In [ ]:
def load_images(image_paths):
    images = []
    raw_images = []
    print("Images:")
    for i, image_path in enumerate([image_paths]):
        print("\t#{}: {}".format(i, image_path))
        
        image, raw_image = preprocess(image_path)
        images.append(image)
        raw_images.append(raw_image)
    return images, raw_images

def get_device(cuda):
    cuda = cuda and torch.cuda.is_available()
    device = torch.device("cuda" if cuda else "cpu")
    if cuda:
        current_device = torch.cuda.current_device()
        print("Device:", torch.cuda.get_device_name(current_device))
    else:
        print("Device: CPU")
    return device

def save_sensitivity(filename, maps):
    maps = maps.cpu().numpy()
    scale = max(maps[maps > 0].max(), -maps[maps <= 0].min())
    maps = maps / scale * 0.5
    maps += 0.5
    maps = cm.bwr_r(maps)[..., :3]
    maps = np.uint8(maps * 255.0)
    maps = cv2.resize(maps, (224, 224), interpolation=cv2.INTER_NEAREST)
    cv2.imwrite(filename, maps)
    
def preprocess(img):
    raw_image = cv2.resize(img, (224, 224))
    image = transforms.Compose(
        [
            transforms.ToTensor(),
            transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
        ]
    )(raw_image[..., ::-1].copy())
    return image

In [ ]:
from tqdm.notebook import tqdm
from collections import Sequence
import torch.nn.functional as F

def occlusion_sensitivity(model, 
                          images, 
                          ids, 
                          mean=None, 
                          patch=35, 
                          stride=1, 
                          n_batches=128):

    torch.set_grad_enabled(False)
    model.eval()
    mean = mean if mean else 0
    patch_H, patch_W = patch if isinstance(patch, Sequence) else (patch, patch)
    pad_H, pad_W = patch_H // 2, patch_W // 2

    # Padded image
    images = F.pad(images, (pad_W, pad_W, pad_H, pad_H), value=mean)
    B, _, H, W = images.shape
    new_H = (H - patch_H) // stride + 1
    new_W = (W - patch_W) // stride + 1

    # Prepare sampling grids
    anchors = []
    grid_h = 0
    while grid_h <= H - patch_H:
        grid_w = 0
        while grid_w <= W - patch_W:
            grid_w += stride
            anchors.append((grid_h, grid_w))
        grid_h += stride

    # Baseline score without occlusion
    baseline = model(images).detach().gather(1, ids)

    # Compute per-pixel logits
    scoremaps = []
    for i in tqdm(range(0, len(anchors), n_batches), leave=False):
        batch_images = []
        batch_ids = []
        for grid_h, grid_w in anchors[i : i + n_batches]:
            images_ = images.clone()
            images_[..., grid_h : grid_h + patch_H, grid_w : grid_w + patch_W] = mean
            batch_images.append(images_)
            batch_ids.append(ids)
        batch_images = torch.cat(batch_images, dim=0)
        batch_ids = torch.cat(batch_ids, dim=0)
        scores = model(batch_images).detach().gather(1, batch_ids)
        scoremaps += list(torch.split(scores, B))

    diffmaps = torch.cat(scoremaps, dim=1) - baseline
    diffmaps = diffmaps.view(B, new_H, new_W)

    return diffmaps

def occlusion_sens_gen(images, model, topk, stride, n_batches):
    
    print("Occlusion Sensitivity:")

    patche_sizes = [10,]# 15, 50]

    images = [torchvision.transforms.ToTensor()(i) for i in images]
    images = torch.stack(images).to(device)
    logits = model(images)
    probs = F.softmax(logits, dim=1)
    probs, ids = probs.sort(dim=1, descending=True)
    res = []
    for i in range(topk):
        for p in patche_sizes:
            print("Patch:", p)
            sensitivity = occlusion_sensitivity(
                model, images, ids[:, [i]], patch=p, stride=stride, n_batches=n_batches
            )
        res.append(sensitivity)
    return res

def topk_img(input_image, cam, k):
  topk = np.argsort(cam.flatten())[-k:]
  topk = np.stack([topk % 224, topk // 224], axis=1)
  out_image = input_image * 255
  out_image[topk[:, 1], topk[:, 0]] = 0
  # assert (out_image != input_image).sum() != 0
  return PIL.Image.fromarray(out_image.reshape(224, 224, 3).astype(np.uint8), 'RGB')

res = occlusion_sens_gen(np.stack(imgs)[:1], model, topk=1, stride=1, n_batches=30)


In [ ]:
(res[0].cpu().numpy() * 255).shape

In [ ]:
img_url = r'https://www.purina.co.uk/sites/default/files/styles/ttt_image_510/public/2020-11/Should%20I%20Get%20a%20Cat%20or%20Dog1.jpg?itok=IdntHkbV'
img_path = './data/dog.jpeg'
urllib.request.urlretrieve(img_url, img_path)

In [ ]:
res, gray_res = layer_cam_gen(model,
                              im_show(img_path),
                              target_layers=[model.features[-1]])

In [ ]:
plt.imshow(res)

EfficientNet

In [ ]:
path = "data/imagenette_sample/"
imagenet_files = os.listdir(path)
for file in imagenet_files:
    #img = transforms(torchvision.io.read_image(path + file)).float()
    res, gray_res = layer_cam_gen(model, im_show(path + file), target_layers=[model.features[-1]])
    f, axarr = plt.subplots(1, 1)
    axarr.imshow(res)
    #plt.imshow(torch.movedim(torchvision.io.read_image(path + file), 0, -1).numpy())
    #plt.imshow(res)
    plt.show()

In [ ]:
for file in imagenet_files:
    #img = transforms(torchvision.io.read_image(path + file)).float()
    res, gray_res = gradpp_cam_gen(model, im_show(path + file), target_layers=[model.features[-1]])
    f, axarr = plt.subplots(1, 1)
    axarr.imshow(res)
    #plt.imshow(torch.movedim(torchvision.io.read_image(path + file), 0, -1).numpy())
    #plt.imshow(res)
    plt.show()

In [ ]:
for file in imagenet_files:
    #img = transforms(torchvision.io.read_image(path + file)).float()
    transform = torchvision.transforms.ToTensor()
    tensor = transform(im_show(path + file)).to(device)
    res = saliency_gen(tensor, model)
    f, axarr = plt.subplots(1, 1)
    axarr.imshow(res.cpu().numpy())
    #plt.imshow(torch.movedim(torchvision.io.read_image(path + file), 0, -1).numpy())
    #plt.imshow(res)
    plt.show()


In [ ]:
np.random.seed(42)
files = np.random.choice(glob("data/imagenette_train/*/*"), 100)
t = torchvision.transforms.ToTensor()
imgs = [im_show(f) for f in files]
'''imgs = torch.stack(imgs)
imgs = torchvision.transforms.Resize((224, 224))(imgs)
gray = []
for f in files:
    res, gray_res = layer_cam_gen(model,
                              im_show(f),
                              target_layers=[model.features[-1]])
    gray.append(gray_res)    '''

In [ ]:
label_dict = {
    "n01440764": 0,
    "n02102040": 217,
    "n02979186": 482,
    "n03000684": 491,
    "n03028079": 497,
    "n03394916": 566,
    "n03417042": 569,
    "n03425413": 571,
    "n03445777": 574,
    "n03888257": 701
}

labels = [label_dict[f.split("\\")[1]]for f in files]

In [ ]:
np.random.seed(42)
gray = []
for img in imgs:
    res, gray_res = guided_backprop_gen(model, img, target_layers=[model.features[-1]])
    gray.append(gray_res)   

transform = torchvision.transforms.ToTensor()
diffs, percs = [], []
for img, g in zip(imgs, gray):
    img = transform(img).unsqueeze(0)
    diff_, perc, cam_ = deletion_game(img.to(device), cam=g, model=model, gray_res=g)
    diffs.append(diff_), percs.append(perc)

diffs_preservation, percs_preservation = [], []
for img in imgs:
    img = transform(img).unsqueeze(0)
    diff_, perc, cam_ = preservation_game(img.to(device), cam=g, model=model, cam_=g)
    diffs_preservation.append(diff_), percs_preservation.append(perc)


In [ ]:
print("===GUIDED BACKPROP===")
print("Deletion game:", np.mean(diffs), np.mean(perc))
print("Preservation game:", np.mean(diffs_preservation), np.mean(percs_preservation))
print("Avg drop:", avg_drop_list([transform(b).unsqueeze(0) for b in imgs], np.array(gray), model, index=labels))
print("Increace in Confidence:", increase_in_confidence_list([transform(b).unsqueeze(0) for b in imgs], np.array(gray), model, labels))
print("Sparsity", sparsity(cam=gray))

In [ ]:
np.random.seed(42)
gray = []
for img in imgs:
    res, gray_res = grad_cam_gen(model, img, target_layers=[model.features[-1]])
    gray.append(gray_res)   

transform = torchvision.transforms.ToTensor()
diffs, percs = [], []
for img, g in zip(imgs, gray):
    img = transform(img).unsqueeze(0)
    diff_, perc, cam_ = deletion_game(img.to(device), cam=g, model=model, gray_res=g)
    diffs.append(diff_), percs.append(perc)

diffs_preservation, percs_preservation = [], []
for img in imgs:
    img = transform(img).unsqueeze(0)
    diff_, perc, cam_ = preservation_game(img.to(device), cam=g, model=model, cam_=g)
    diffs_preservation.append(diff_), percs_preservation.append(perc)


In [ ]:
diff_, perc, cam_ = deletion_game(img.to(device), cam=g, model=model, gray_res=g)

In [ ]:
print("===GRAD CAM===")
print("Deletion game:", np.mean(diffs), np.mean(perc))
print("Preservation game:", np.mean(diffs_preservation), np.mean(percs_preservation))
print("Avg drop:", avg_drop_list([transform(b).unsqueeze(0) for b in imgs], np.array(gray), model, index=labels))
print("Increace in Confidence:", increase_in_confidence_list([transform(b).unsqueeze(0) for b in imgs], np.array(gray), model, labels))
print("Sparsity", sparsity(cam=gray))

Deletion Game

In [ ]:
transform = torchvision.transforms.Resize((224, 224))
diffs, percs = [], []
for f, g in zip(files, gray):
    img = torchvision.io.read_image(f, mode=torchvision.io.ImageReadMode.RGB).float().unsqueeze(0)
    img = transform(img)
    diff_, perc, cam_ = deletion_game(img.to(device), cam=g, model=model, gray_res=g)
    diffs.append(diff_), percs.append(perc)

In [ ]:
np.mean(diffs), np.mean(perc)

Preservation Game

In [ ]:
transform = torchvision.transforms.Resize((224, 224))
diffs_preservation, percs_preservation = [], []
for f, g in zip(files, gray):
    img = torchvision.io.read_image(f, mode=torchvision.io.ImageReadMode.RGB).float().unsqueeze(0)
    img = transform(img)
    diff_, perc, cam_ = preservation_game(img.to(device), cam=g, model=model, cam_=g)
    diffs_preservation.append(diff_), percs_preservation.append(perc)

In [ ]:
np.mean(diffs_preservation), np.mean(percs_preservation)

Average Drop

In [ ]:
label_dict = {
    "n01440764": 0,
    "n02102040": 217,
    "n02979186": 482,
    "n03000684": 491,
    "n03028079": 497,
    "n03394916": 566,
    "n03417042": 569,
    "n03425413": 571,
    "n03445777": 574,
    "n03888257": 701
}

labels = [label_dict[f.split("\\")[1]]for f in files]

In [ ]:
avg_drop_list(imgs.unsqueeze(1), np.array(gray), model, index=labels)

Increase in Confidence

In [ ]:
increase_in_confidence_list(imgs.unsqueeze(1), np.array(gray), model, labels)

Sparsity 

In [ ]:
print(f" Sparsity = {sparsity(cam=gray)}")

VGG

In [ ]:
model = models.vgg16(weights='DEFAULT').to(device)

In [ ]:
list_iou = []

for i in range(len(data["a"])):
  name_dict = data["a"][i]["data_row"]["external_id"]
  
  bbox_dict_i = data["a"][i]["projects"][list(data["a"][i]["projects"].keys())[0]]["labels"][0]["annotations"]["objects"][0]["bounding_box"]
  bb_top_i = bbox_dict_i["top"]
  bb_left_i = bbox_dict_i["left"]
  bb_height_i = bbox_dict_i["height"]
  bb_width_i = bbox_dict_i["width"]
  im = Image.open(f"data_labeling_imagenet/{name_dict}")
  res, gray_res = guided_backprop_gen(model, 
                               im_show(f"data_labeling_imagenet/{name_dict}"),
                               target_layers= [model.features[-1]])
  bb_arr = np.array([bb_top_i, bb_left_i, bb_top_i + bb_height_i, bb_left_i + bb_width_i])
  iou_i = iou_loc(bb_arr, gray_res)
  list_iou.append(iou_i)

In [ ]:
print(f"IOU Average: {np.mean(list_iou)}")

In [ ]:
np.random.seed(42)
gray = []
for img in imgs:
    res, gray_res = guided_backprop_gen(model, img, target_layers=[model.features[-1]])
    gray.append(gray_res)   

transform = torchvision.transforms.ToTensor()
diffs, percs = [], []
for img, g in zip(imgs, gray):
    img = transform(img).unsqueeze(0)
    diff_, perc, cam_ = deletion_game(img.to(device), cam=g, model=model, gray_res=g)
    diffs.append(diff_), percs.append(perc)

diffs_preservation, percs_preservation = [], []
for img in imgs:
    img = transform(img).unsqueeze(0)
    diff_, perc, cam_ = preservation_game(img.to(device), cam=g, model=model, cam_=g)
    diffs_preservation.append(diff_), percs_preservation.append(perc)


In [ ]:
print("===GUIDED BACKPROP===")
print("Deletion game:", np.mean(diffs), np.mean(perc))
print("Preservation game:", np.mean(diffs_preservation), np.mean(percs_preservation))
print("Avg drop:", avg_drop_list([transform(b).unsqueeze(0) for b in imgs], np.array(gray), model, index=labels))
print("Increace in Confidence:", increase_in_confidence_list([transform(b).unsqueeze(0) for b in imgs], np.array(gray), model, labels))
print("Sparsity", sparsity(cam=gray))

In [ ]:
np.random.seed(42)
gray = []
for img in imgs:
    res, gray_res = grad_cam_gen(model, img, target_layers=[model.features[-1]])
    gray.append(gray_res)   

transform = torchvision.transforms.ToTensor()
diffs, percs = [], []
for img, g in zip(imgs, gray):
    img = transform(img).unsqueeze(0)
    diff_, perc, cam_ = deletion_game(img.to(device), cam=g, model=model, gray_res=g)
    diffs.append(diff_), percs.append(perc)

diffs_preservation, percs_preservation = [], []
for img in imgs:
    img = transform(img).unsqueeze(0)
    diff_, perc, cam_ = preservation_game(img.to(device), cam=g, model=model, cam_=g)
    diffs_preservation.append(diff_), percs_preservation.append(perc)


In [ ]:
print("===GRAD CAM===")
print("Deletion game:", np.mean(diffs), np.mean(perc))
print("Preservation game:", np.mean(diffs_preservation), np.mean(percs_preservation))
print("Avg drop:", avg_drop_list([transform(b).unsqueeze(0) for b in imgs], np.array(gray), model, index=labels))
print("Increace in Confidence:", increase_in_confidence_list([transform(b).unsqueeze(0) for b in imgs], np.array(gray), model, labels))
print("Sparsity", sparsity(cam=gray))

In [ ]:
path = "data/imagenette_sample/"
imagenet_files = os.listdir(path)
for file in imagenet_files:
    #img = transforms(torchvision.io.read_image(path + file)).float()
    res, gray_res = layer_cam_gen(model, im_show(path + file), target_layers=[model.avgpool])
    f, axarr = plt.subplots(1, 1)
    axarr.imshow(res)
    #plt.imshow(torch.movedim(torchvision.io.read_image(path + file), 0, -1).numpy())
    #plt.imshow(res)
    plt.show()

In [ ]:
for file in imagenet_files:
    #img = transforms(torchvision.io.read_image(path + file)).float()
    res, gray_res = gradpp_cam_gen(model, im_show(path + file), target_layers=[model.features[-1]])
    f, axarr = plt.subplots(1, 1)
    axarr.imshow(res)
    #plt.imshow(torch.movedim(torchvision.io.read_image(path + file), 0, -1).numpy())
    #plt.imshow(res)
    plt.show()

In [ ]:
for file in imagenet_files:
    #img = transforms(torchvision.io.read_image(path + file)).float()
    transform = torchvision.transforms.ToTensor()
    tensor = transform(im_show(path + file)).to(device)
    res = saliency_gen(tensor, model)
    f, axarr = plt.subplots(1, 1)
    axarr.imshow(res.cpu().numpy())
    #plt.imshow(torch.movedim(torchvision.io.read_image(path + file), 0, -1).numpy())
    #plt.imshow(res)
    plt.show()

In [ ]:
np.random.seed(42)
files = np.random.choice(glob("data/imagenette_train/*/*"), 100)
t = torchvision.transforms.ToTensor()
imgs = [t(im_show(f)) for f in files]
imgs = torch.stack(imgs)
imgs = torchvision.transforms.Resize((224, 224))(imgs)
gray = []
for f in files:
    res, gray_res = layer_cam_gen(model,
                              im_show(f),
                              target_layers=[model.features[-1]])
    gray.append(gray_res)    

Deletion Game

In [ ]:
transform = torchvision.transforms.Resize((224, 224))
diffs, percs = [], []
for f, g in zip(files, gray):
    img = torchvision.io.read_image(f, mode=torchvision.io.ImageReadMode.RGB).float().unsqueeze(0)
    img = transform(img)
    diff_, perc, cam_ = deletion_game(img.to(device), cam=g, model=model, gray_res=g)
    diffs.append(diff_), percs.append(perc)

In [ ]:
np.mean(diffs), np.mean(perc)

Preservation Game

In [ ]:
transform = torchvision.transforms.Resize((224, 224))
diffs_preservation, percs_preservation = [], []
for f, g in zip(files, gray):
    img = torchvision.io.read_image(f, mode=torchvision.io.ImageReadMode.RGB).float().unsqueeze(0)
    img = transform(img)
    diff_, perc, cam_ = preservation_game(img.to(device), cam=g, model=model, cam_=g)
    diffs_preservation.append(diff_), percs_preservation.append(perc)

In [ ]:
np.mean(diffs_preservation), np.mean(percs_preservation)

Average Drop

In [ ]:
label_dict = {
    "n01440764": 0,
    "n02102040": 217,
    "n02979186": 482,
    "n03000684": 491,
    "n03028079": 497,
    "n03394916": 566,
    "n03417042": 569,
    "n03425413": 571,
    "n03445777": 574,
    "n03888257": 701
}

labels = [label_dict[f.split("\\")[1]]for f in files]

In [ ]:
avg_drop_list(imgs.unsqueeze(1), np.array(gray), model, index=labels)

Increace in Confidence

In [ ]:
increase_in_confidence_list(imgs.unsqueeze(1), np.array(gray), model, labels)

Sparsity

In [ ]:
print(f" Sparsity = {sparsity(cam=gray)}")

Resnet

In [ ]:
model = models.resnet18(weights="DEFAULT")

In [ ]:
list_iou = []

for i in range(len(data["a"])):
  name_dict = data["a"][i]["data_row"]["external_id"]
  
  bbox_dict_i = data["a"][i]["projects"][list(data["a"][i]["projects"].keys())[0]]["labels"][0]["annotations"]["objects"][0]["bounding_box"]
  bb_top_i = bbox_dict_i["top"]
  bb_left_i = bbox_dict_i["left"]
  bb_height_i = bbox_dict_i["height"]
  bb_width_i = bbox_dict_i["width"]
  im = Image.open(f"data_labeling_imagenet/{name_dict}")
  res, gray_res = guided_backprop_gen(model, 
                               im_show(f"data_labeling_imagenet/{name_dict}"),
                               target_layers= [model.layer4])
  bb_arr = np.array([bb_top_i, bb_left_i, bb_top_i + bb_height_i, bb_left_i + bb_width_i])
  iou_i = iou_loc(bb_arr, gray_res)
  list_iou.append(iou_i)

In [ ]:
print(f"IOU Average: {np.mean(list_iou)}")

In [ ]:
np.random.seed(42)
gray = []
for img in imgs:
    res, gray_res = guided_backprop_gen(model, img, target_layers=[model.layer4])
    gray.append(gray_res)   

transform = torchvision.transforms.ToTensor()
diffs, percs = [], []
for img, g in zip(imgs, gray):
    img = transform(img).unsqueeze(0)
    diff_, perc, cam_ = deletion_game(img.to(device), cam=g, model=model, gray_res=g)
    diffs.append(diff_), percs.append(perc)

diffs_preservation, percs_preservation = [], []
for img in imgs:
    img = transform(img).unsqueeze(0)
    diff_, perc, cam_ = preservation_game(img.to(device), cam=g, model=model, cam_=g)
    diffs_preservation.append(diff_), percs_preservation.append(perc)


In [ ]:
print("===GUIDED BACKPROP===")
print("Deletion game:", np.mean(diffs), np.mean(perc))
print("Preservation game:", np.mean(diffs_preservation), np.mean(percs_preservation))
print("Avg drop:", avg_drop_list([transform(b).unsqueeze(0) for b in imgs], np.array(gray), model, index=labels))
print("Increace in Confidence:", increase_in_confidence_list([transform(b).unsqueeze(0) for b in imgs], np.array(gray), model, labels))
print("Sparsity", sparsity(cam=gray))

In [ ]:
np.random.seed(42)
gray = []
for img in imgs:
    res, gray_res = grad_cam_gen(model, img, target_layers=[model.layer4])
    gray.append(gray_res)   

transform = torchvision.transforms.ToTensor()
diffs, percs = [], []
for img, g in zip(imgs, gray):
    img = transform(img).unsqueeze(0)
    diff_, perc, cam_ = deletion_game(img.to(device), cam=g, model=model, gray_res=g)
    diffs.append(diff_), percs.append(perc)

diffs_preservation, percs_preservation = [], []
for img in imgs:
    img = transform(img).unsqueeze(0)
    diff_, perc, cam_ = preservation_game(img.to(device), cam=g, model=model, cam_=g)
    diffs_preservation.append(diff_), percs_preservation.append(perc)


In [ ]:
print("===GRAD CAM===")
print("Deletion game:", np.mean(diffs), np.mean(perc))
print("Preservation game:", np.mean(diffs_preservation), np.mean(percs_preservation))
print("Avg drop:", avg_drop_list([transform(b).unsqueeze(0) for b in imgs], np.array(gray), model, index=labels))
print("Increace in Confidence:", increase_in_confidence_list([transform(b).unsqueeze(0) for b in imgs], np.array(gray), model, labels))
print("Sparsity", sparsity(cam=gray))

In [ ]:
path = "data/imagenette_sample/"
imagenet_files = os.listdir(path)
for file in imagenet_files:
    #img = transforms(torchvision.io.read_image(path + file)).float()
    res, gray_res = layer_cam_gen(model, im_show(
        path + file), target_layers=[model.layer4])
    f, axarr = plt.subplots(1, 1)
    axarr.imshow(res)
    #plt.imshow(torch.movedim(torchvision.io.read_image(path + file), 0, -1).numpy())
    # plt.imshow(res)
    plt.show()

In [ ]:
for file in imagenet_files:
    #img = transforms(torchvision.io.read_image(path + file)).float()
    res, gray_res = gradpp_cam_gen(model, im_show(
        path + file), target_layers=[model.layer4])
    f, axarr = plt.subplots(1, 1)
    axarr.imshow(res)
    #plt.imshow(torch.movedim(torchvision.io.read_image(path + file), 0, -1).numpy())
    # plt.imshow(res)
    plt.show()

In [ ]:
for file in imagenet_files:
    #img = transforms(torchvision.io.read_image(path + file)).float()
    transform = torchvision.transforms.ToTensor()
    tensor = transform(im_show(path + file)).to(device)
    res = saliency_gen(tensor, model)
    f, axarr = plt.subplots(1, 1)
    axarr.imshow(res.cpu().numpy())
    #plt.imshow(torch.movedim(torchvision.io.read_image(path + file), 0, -1).numpy())
    # plt.imshow(res)
    plt.show()

In [ ]:
np.random.seed(42)
files = np.random.choice(glob("data/imagenette_train/*/*"), 100)
t = torchvision.transforms.ToTensor()
imgs = [t(im_show(f)) for f in files]
imgs = torch.stack(imgs)
imgs = torchvision.transforms.Resize((224, 224))(imgs)
gray = []
for f in files:
    res, gray_res = layer_cam_gen(model,
                              im_show(f),
                              target_layers=[model.layer4])
    gray.append(gray_res)    

Deletion Game

In [ ]:
transform = torchvision.transforms.Resize((224, 224))
diffs, percs = [], []
for f, g in zip(files, gray):
    img = torchvision.io.read_image(f, mode=torchvision.io.ImageReadMode.RGB).float().unsqueeze(0)
    img = transform(img)
    diff_, perc, cam_ = deletion_game(img.to(device), cam=g, model=model, gray_res=g)
    diffs.append(diff_), percs.append(perc)

In [ ]:
np.mean(diffs), np.mean(perc)

Preservation Game

In [ ]:
transform = torchvision.transforms.Resize((224, 224))
diffs_preservation, percs_preservation = [], []
for f, g in zip(files, gray):
    img = torchvision.io.read_image(f, mode=torchvision.io.ImageReadMode.RGB).float().unsqueeze(0)
    img = transform(img)
    diff_, perc, cam_ = preservation_game(img.to(device), cam=g, model=model, cam_=g)
    diffs_preservation.append(diff_), percs_preservation.append(perc)

In [ ]:
np.mean(diffs_preservation), np.mean(percs_preservation)

Average Drop

In [ ]:
label_dict = {
    "n01440764": 0,
    "n02102040": 217,
    "n02979186": 482,
    "n03000684": 491,
    "n03028079": 497,
    "n03394916": 566,
    "n03417042": 569,
    "n03425413": 571,
    "n03445777": 574,
    "n03888257": 701
}

labels = [label_dict[f.split("\\")[1]]for f in files]

In [ ]:
avg_drop_list(imgs.unsqueeze(1), np.array(gray), model, index=labels)

Increase in Confidence

In [ ]:
increase_in_confidence_list(imgs.unsqueeze(1), np.array(gray), model, labels)

Sparsity

In [ ]:
print(f" Sparsity = {sparsity(cam=gray)}")